# 0.Setup and data loading

In [1]:
import pandas as pd
import numpy as np
import os
import random

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

import importlib
import train_models_util
import feature_importance
import evaluate_models_util
importlib.reload(train_models_util)
importlib.reload(feature_importance)
importlib.reload(evaluate_models_util)
from train_models_util import (
    train_xgb_optuna, 
    train_catboost_optuna, 
    train_lgbm_optuna,
    train_ensemble
)
from evaluate_models_util import evaluate_and_log
from feature_importance import get_top_features_shap

# set all random seeds for reproducibility
np.random.seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)

In [2]:
EXPERIMENT_NAME = "Fraud-detection-custom-thresholds_v10_downsample_auc_pr"

In [3]:
data_folder = 'ieee-fraud-detection-data/processed/'

data = pd.read_csv(f'{data_folder}train_processed.csv')

In [4]:
data = data.drop(columns=['TransactionDT','uid'], axis='columns')

In [5]:
cat_cols = np.array(data.select_dtypes(include=['object']).columns)
for c in cat_cols:
    if c in data.columns:
        if data[c].isnull().sum()>0:
            data[c].fillna('missing', inplace=True)
        data[c] = data[c].astype('category')

In [6]:
# attempt without suffle, since the data have time relation
n = len(data)
split_idx = int(n * 0.8) 

train = data.iloc[:split_idx]
test = data.iloc[split_idx:]

In [7]:
del data, cat_cols, c #,fraud_indices, non_fraud_indices,
import gc
gc.collect()

0

## Some more features

In [8]:
NON_FRAUD_RATIO = 5   # keep 5 non-fraud per fraud
RANDOM_STATE = 42

#Save original index
train["original_index"] = train.index

fraud_df = train[train['isFraud'] == 1]
non_fraud_df = train[train['isFraud'] == 0]

# -----------------------------
# COMPUTE SAMPLE SIZE
# -----------------------------
target_non_fraud = min(
    len(non_fraud_df),
    NON_FRAUD_RATIO * len(fraud_df)
)

# -----------------------------
# DOWNSAMPLE
# -----------------------------
downsampled_non_fraud = non_fraud_df.sample(
    n=target_non_fraud,
    random_state=RANDOM_STATE
)

train = pd.concat([fraud_df, downsampled_non_fraud], ignore_index=True)

# Restore original order
train = train.sort_values("original_index")

# Optional: drop helper column
train = train.drop(columns=["original_index"])

print("Done.")
print(f"Fraud kept: {len(fraud_df)}")
print(f"Non-fraud kept: {len(downsampled_non_fraud)}")
print(f"Final size: {len(train)}")

Done.
Fraud kept: 16599
Non-fraud kept: 82995
Final size: 99594


In [9]:
del downsampled_non_fraud, target_non_fraud, fraud_df, non_fraud_df
import gc
gc.collect()

0

## 1. Full Features

In [10]:
# split to X and y
X_train = train.drop(columns='isFraud', axis='columns')
X_test = test.drop(columns='isFraud', axis='columns')

y_train = train['isFraud']
y_test = test['isFraud']

del train, test

In [11]:
X_train.shape, X_test.shape

((99594, 748), (118108, 748))

### 1.1. XGBoost

In [12]:
model_xgboost_full, best_params_xgboost_full, hist_df, plot_paths = train_xgb_optuna(X_train, y_train)

[I 2025-12-30 18:58:55,763] A new study created in memory with name: xgboost_aucpr_optimization


[Optuna XGBoost Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-12-30 19:00:48,955] Trial 0 finished with value: 0.8815459377692526 and parameters: {'learning_rate': 0.017021382657989494, 'max_depth': 3, 'subsample': 0.9852287777612144, 'colsample_bytree': 0.8082455118398606, 'min_child_weight': 0.0023692989617083455, 'gamma': 2.9760304497265047}. Best is trial 0 with value: 0.8815459377692526.
[I 2025-12-30 19:02:38,171] Trial 1 finished with value: 0.8930147183926035 and parameters: {'learning_rate': 0.020903924756192904, 'max_depth': 4, 'subsample': 0.7624407339735624, 'colsample_bytree': 0.6819803313362081, 'min_child_weight': 0.031187161159167146, 'gamma': 4.1658634185980405}. Best is trial 1 with value: 0.8930147183926035.
[I 2025-12-30 19:06:05,778] Trial 2 finished with value: 0.9123931918746532 and parameters: {'learning_rate': 0.015715452762831466, 'max_depth': 8, 'subsample': 0.9368563854878987, 'colsample_bytree': 0.8386623202640249, 'min_child_weight': 0.004433503557765655, 'gamma': 0.2861741006413454}. Best is trial 2 with val

[INFO] Best XGBoost params: {'learning_rate': 0.02614373405827699, 'max_depth': 8, 'subsample': 0.855847752175603, 'colsample_bytree': 0.6215883337426336, 'min_child_weight': 0.05377545923860718, 'gamma': 1.4778062360758693}
[0]	validation_0-auc:0.80400
[100]	validation_0-auc:0.92449
[200]	validation_0-auc:0.93423
[300]	validation_0-auc:0.93943
[400]	validation_0-auc:0.94153
[489]	validation_0-auc:0.94101


In [13]:
evaluate_and_log(model_xgboost_full, X_test, y_test, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths)

2025/12/30 20:16:50 INFO mlflow.tracking.fluent: Experiment with name 'Fraud-detection-custom-thresholds_v10_downsample_auc_pr' does not exist. Creating a new experiment.


[INFO] Logged metrics: {'roc_auc': np.float64(0.9078093198919356), 'pr_auc': np.float64(0.5082376970257724), 'precision': 0.26443909663087745, 'recall': 0.703001968503937, 'f1': 0.3843153080441216, 'custom_loss': np.float64(1.0892318894571071)}
[INFO] Logged SHAP summary plot.


2025/12/30 20:17:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9078093198919356),
 'pr_auc': np.float64(0.5082376970257724),
 'precision': 0.26443909663087745,
 'recall': 0.703001968503937,
 'f1': 0.3843153080441216,
 'custom_loss': np.float64(1.0892318894571071)}

### 1.2. LightGBM

In [14]:
model_lgbm_full, best_params_lgbm_full, hist_df, plot_paths = train_lgbm_optuna(X_train, y_train)

[I 2025-12-30 20:17:10,194] A new study created in memory with name: lgbm_aucpr_optimization


[Optuna LightGBM Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-12-30 20:19:15,803] Trial 0 finished with value: 0.9303887134433104 and parameters: {'num_leaves': 438, 'max_depth': 18, 'learning_rate': 0.024813227797543944, 'min_child_samples': 78, 'reg_alpha': 0.11131267825429031, 'reg_lambda': 0.14680824732832343, 'colsample_bytree': 0.9597702335985017, 'subsample': 0.8196496398178817}. Best is trial 0 with value: 0.9303887134433104.
[I 2025-12-30 20:20:19,584] Trial 1 finished with value: 0.9255921743025621 and parameters: {'num_leaves': 187, 'max_depth': 10, 'learning_rate': 0.02773062299371437, 'min_child_samples': 96, 'reg_alpha': 0.17818292354755488, 'reg_lambda': 0.25870408024126584, 'colsample_bytree': 0.8545941107779105, 'subsample': 0.8592687410563851}. Best is trial 0 with value: 0.9303887134433104.
[I 2025-12-30 20:22:10,837] Trial 2 finished with value: 0.9268491523795716 and parameters: {'num_leaves': 169, 'max_depth': 14, 'learning_rate': 0.014524914077023918, 'min_child_samples': 27, 'reg_alpha': 0.37364631726463815, 'reg_l

[INFO] Best LightGBM params: {'num_leaves': 395, 'max_depth': 19, 'learning_rate': 0.02082403523122257, 'min_child_samples': 63, 'reg_alpha': 0.3412339828667133, 'reg_lambda': 0.07514268739421674, 'colsample_bytree': 0.8297046496536533, 'subsample': 0.9057142544451396}
[50]	valid_0's auc: 0.939185	valid_0's binary_logloss: 0.285796
[100]	valid_0's auc: 0.9453	valid_0's binary_logloss: 0.241926
[150]	valid_0's auc: 0.946967	valid_0's binary_logloss: 0.221692
[200]	valid_0's auc: 0.946976	valid_0's binary_logloss: 0.211693
[250]	valid_0's auc: 0.947573	valid_0's binary_logloss: 0.204914
[300]	valid_0's auc: 0.948087	valid_0's binary_logloss: 0.200582
[350]	valid_0's auc: 0.948253	valid_0's binary_logloss: 0.1974
[400]	valid_0's auc: 0.948385	valid_0's binary_logloss: 0.195038
[450]	valid_0's auc: 0.948673	valid_0's binary_logloss: 0.194003
[500]	valid_0's auc: 0.949212	valid_0's binary_logloss: 0.193806
[550]	valid_0's auc: 0.949315	valid_0's binary_logloss: 0.194674


In [15]:
evaluate_and_log(model_lgbm_full,  X_test, y_test, experiment_name=EXPERIMENT_NAME, run_name="LightGBM_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9167937573548764), 'pr_auc': np.float64(0.5812767824950087), 'precision': 0.413591285969761, 'recall': 0.6259842519685039, 'f1': 0.49809104258443465, 'custom_loss': np.float64(1.3174975446201782)}
[INFO] Logged SHAP summary plot.


2025/12/30 21:21:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9167937573548764),
 'pr_auc': np.float64(0.5812767824950087),
 'precision': 0.413591285969761,
 'recall': 0.6259842519685039,
 'f1': 0.49809104258443465,
 'custom_loss': np.float64(1.3174975446201782)}

### 1.3. CatBoost

In [16]:
model_catboost_full, best_params_catboost_full, hist_df, plot_paths = train_catboost_optuna(X_train, y_train)

[I 2025-12-30 21:21:38,471] A new study created in memory with name: catboost_aucpr_optimization


[Optuna CatBoost Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-12-30 21:27:45,584] Trial 0 finished with value: 0.9005063646683223 and parameters: {'learning_rate': 0.012114413629312287, 'depth': 8, 'l2_leaf_reg': 4.532958958252056, 'subsample': 0.8250834668067653, 'border_count': 63}. Best is trial 0 with value: 0.9005063646683223.
[I 2025-12-30 21:30:55,046] Trial 1 finished with value: 0.8886750398904425 and parameters: {'learning_rate': 0.017430238582181155, 'depth': 5, 'l2_leaf_reg': 8.437959647535058, 'subsample': 0.6635671917917999, 'border_count': 102}. Best is trial 0 with value: 0.9005063646683223.
[I 2025-12-30 21:33:33,661] Trial 2 finished with value: 0.8885071796802295 and parameters: {'learning_rate': 0.02309646967616278, 'depth': 4, 'l2_leaf_reg': 8.26236300691355, 'subsample': 0.8399605630074497, 'border_count': 102}. Best is trial 0 with value: 0.9005063646683223.
[I 2025-12-30 21:35:35,094] Trial 3 finished with value: 0.8705681367037098 and parameters: {'learning_rate': 0.013109543612431602, 'depth': 3, 'l2_leaf_reg': 5

[INFO] Best CatBoost params: {'learning_rate': 0.027258338352952993, 'depth': 8, 'l2_leaf_reg': 4.802904112680188, 'subsample': 0.8400577607870461, 'border_count': 98}
0:	test: 0.8444725	best: 0.8444725 (0)	total: 573ms	remaining: 47m 43s
100:	test: 0.9048816	best: 0.9048816 (100)	total: 56.8s	remaining: 45m 53s
200:	test: 0.9189296	best: 0.9189296 (200)	total: 1m 52s	remaining: 44m 42s
300:	test: 0.9259529	best: 0.9259529 (300)	total: 2m 47s	remaining: 43m 34s
400:	test: 0.9320085	best: 0.9320085 (400)	total: 3m 42s	remaining: 42m 27s
500:	test: 0.9372334	best: 0.9372334 (500)	total: 4m 37s	remaining: 41m 35s
600:	test: 0.9415142	best: 0.9415142 (600)	total: 5m 33s	remaining: 40m 42s
700:	test: 0.9435147	best: 0.9435147 (700)	total: 6m 29s	remaining: 39m 45s
800:	test: 0.9453995	best: 0.9453995 (800)	total: 7m 24s	remaining: 38m 47s
900:	test: 0.9463297	best: 0.9463964 (894)	total: 8m 18s	remaining: 37m 49s
1000:	test: 0.9464893	best: 0.9467097 (954)	total: 9m 13s	remaining: 36m 49s
1

In [17]:
evaluate_and_log(model_catboost_full,  X_test, y_test, experiment_name=EXPERIMENT_NAME, run_name="CATboost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9156110566318236), 'pr_auc': np.float64(0.5248011941207337), 'precision': 0.2635207736389685, 'recall': 0.7241633858267716, 'f1': 0.3864233193277311, 'custom_loss': np.float64(1.0187709553967554)}
[INFO] Logged SHAP summary plot.


2025/12/31 00:02:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9156110566318236),
 'pr_auc': np.float64(0.5248011941207337),
 'precision': 0.2635207736389685,
 'recall': 0.7241633858267716,
 'f1': 0.3864233193277311,
 'custom_loss': np.float64(1.0187709553967554)}

## 2. Reduced Features

In [18]:
# ---- 1. Compute SHAP importance for each model ----
df_xgb = get_top_features_shap(model_xgboost_full, X_train, y_train).rename(
    columns={"importance": "importance_xgb"}
)
df_lgbm = get_top_features_shap(model_lgbm_full, X_train, y_train).rename(
    columns={"importance": "importance_lgbm"}
)
df_cat = get_top_features_shap(model_catboost_full, X_train, y_train).rename(
    columns={"importance": "importance_cat"}
)

# ---- 2. Merge all importance tables ----
df = (
    df_xgb.merge(df_lgbm, on="feature")
          .merge(df_cat, on="feature")
)

# ---- 3. Rank features within each model (1 = most important) ----
df["rank_xgb"] = df["importance_xgb"].rank(method="min", ascending=False)
df["rank_lgbm"] = df["importance_lgbm"].rank(method="min", ascending=False)
df["rank_cat"] = df["importance_cat"].rank(method="min", ascending=False)

# ---- 4. Determine the top 30% threshold ----
n_features = len(df)
top_30_cutoff = int(n_features * 0.30)

# ---- 5. Check if a feature appears in top 30% in each model ----
df["in_top_xgb"]  = df["rank_xgb"]  <= top_30_cutoff
df["in_top_lgbm"] = df["rank_lgbm"] <= top_30_cutoff
df["in_top_cat"]  = df["rank_cat"]  <= top_30_cutoff

# ---- 6. Apply the rule: “Top 30% for at least 2 models” ----
df["top_count"] = (
    df["in_top_xgb"].astype(int)
  + df["in_top_lgbm"].astype(int)
  + df["in_top_cat"].astype(int)
)

df_selected = df[df["top_count"] >= 2]  # final selected features


In [19]:
# ---- 7. Sort by mean importance (optional but nice) ----
df_selected["importance_mean"] = df_selected[["importance_xgb","importance_lgbm","importance_cat"]].mean(axis=1)
df_selected = df_selected.sort_values("importance_mean", ascending=False)

# ---- 8. Save to CSV ----
df.to_csv(f"{data_folder}/feature_importances_shap_fullmodels.csv", index=False)
df_selected.to_csv(f"{data_folder}/selected_features_shap.csv", index=False)

# ---- 9. List of selected feature names ----
selected_feature_list = df_selected["feature"].tolist()

In [20]:
del df_xgb, df_lgbm, df_cat, df_selected, df

In [21]:
X_train_reduced = X_train[selected_feature_list]
X_test_reduced = X_test[selected_feature_list]

In [22]:
X_train_reduced.shape, X_test_reduced.shape

((99594, 216), (118108, 216))

### 2.1. XGBoost 

In [23]:
model_xgboost_reduced, best_params_xgboost_reduced, hist_df, plot_paths = train_xgb_optuna(X_train_reduced, y_train)

[I 2025-12-31 00:07:54,587] A new study created in memory with name: xgboost_aucpr_optimization


[Optuna XGBoost Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-12-31 00:08:27,736] Trial 0 finished with value: 0.8840300798264623 and parameters: {'learning_rate': 0.02092465072355276, 'max_depth': 3, 'subsample': 0.9771946777450753, 'colsample_bytree': 0.5656754791931622, 'min_child_weight': 0.005815968018219665, 'gamma': 4.888205320471238}. Best is trial 0 with value: 0.8840300798264623.
[I 2025-12-31 00:08:59,027] Trial 1 finished with value: 0.8744530426182887 and parameters: {'learning_rate': 0.010944514565163984, 'max_depth': 5, 'subsample': 0.5010085876492529, 'colsample_bytree': 0.6174297085202642, 'min_child_weight': 0.343515236690497, 'gamma': 4.104350968914654}. Best is trial 0 with value: 0.8840300798264623.
[I 2025-12-31 00:09:57,549] Trial 2 finished with value: 0.9152509699164422 and parameters: {'learning_rate': 0.02136454653807071, 'max_depth': 8, 'subsample': 0.7308527466178657, 'colsample_bytree': 0.7399973854356304, 'min_child_weight': 0.007314934845446755, 'gamma': 1.6574590959216307}. Best is trial 2 with value: 0.91

[INFO] Best XGBoost params: {'learning_rate': 0.029731259226139835, 'max_depth': 8, 'subsample': 0.7018247454042068, 'colsample_bytree': 0.6052063340243405, 'min_child_weight': 0.0391272419611524, 'gamma': 3.941289846852767}
[0]	validation_0-auc:0.79139
[100]	validation_0-auc:0.92412
[200]	validation_0-auc:0.93387
[300]	validation_0-auc:0.93770
[400]	validation_0-auc:0.93854
[500]	validation_0-auc:0.93937
[600]	validation_0-auc:0.93958
[670]	validation_0-auc:0.93933


In [24]:
evaluate_and_log(model_xgboost_reduced, X_test_reduced, y_test, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths)
evaluate_and_log(model_xgboost_reduced, X_test_reduced, y_test, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.10)
evaluate_and_log(model_xgboost_reduced, X_test_reduced, y_test, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.02)

[INFO] Logged metrics: {'roc_auc': np.float64(0.904637266202615), 'pr_auc': np.float64(0.5108799716054857), 'precision': 0.30010893246187365, 'recall': 0.6779035433070866, 'f1': 0.41603745092117184, 'custom_loss': np.float64(1.1627070139194635)}
[INFO] Logged SHAP summary plot.


2025/12/31 00:31:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.
[INFO] Logged metrics: {'roc_auc': np.float64(0.904637266202615), 'pr_auc': np.float64(0.5108799716054857), 'precision': 0.08481748851524135, 'recall': 0.922244094488189, 'f1': 0.1553478540194392, 'custom_loss': np.float64(0.609958681884377)}
[INFO] Logged SHAP summary plot.


2025/12/31 00:31:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.
[INFO] Logged metrics: {'roc_auc': np.float64(0.904637266202615), 'pr_auc': np.float64(0.5108799716054857), 'precision': 0.04170511744966443, 'recall': 0.9785925196850394, 'f1': 0.08000080463469585, 'custom_loss': np.float64(0.8473854438310698)}
[INFO] Logged SHAP summary plot.


2025/12/31 00:31:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.904637266202615),
 'pr_auc': np.float64(0.5108799716054857),
 'precision': 0.04170511744966443,
 'recall': 0.9785925196850394,
 'f1': 0.08000080463469585,
 'custom_loss': np.float64(0.8473854438310698)}

### 2.2. LightGBM

In [25]:
model_lgmb_reduced, best_params_lgmb_reduced, hist_df, plot_paths = train_lgbm_optuna(X_train_reduced, y_train)

[I 2025-12-31 00:31:58,775] A new study created in memory with name: lgbm_aucpr_optimization


[Optuna LightGBM Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-12-31 00:32:46,383] Trial 0 finished with value: 0.9295967026866888 and parameters: {'num_leaves': 425, 'max_depth': 15, 'learning_rate': 0.013576591301195238, 'min_child_samples': 95, 'reg_alpha': 0.3388882132452687, 'reg_lambda': 0.39325075045344976, 'colsample_bytree': 0.9943244950065704, 'subsample': 0.9813244209581085}. Best is trial 0 with value: 0.9295967026866888.
[I 2025-12-31 00:33:40,829] Trial 1 finished with value: 0.9302020173691249 and parameters: {'num_leaves': 177, 'max_depth': 19, 'learning_rate': 0.015252554304863395, 'min_child_samples': 32, 'reg_alpha': 0.0010823805508800577, 'reg_lambda': 0.09139958489534267, 'colsample_bytree': 0.9390925726586566, 'subsample': 0.8117742652508098}. Best is trial 1 with value: 0.9302020173691249.
[I 2025-12-31 00:34:29,104] Trial 2 finished with value: 0.93200491708255 and parameters: {'num_leaves': 152, 'max_depth': 19, 'learning_rate': 0.015217817301396357, 'min_child_samples': 81, 'reg_alpha': 0.4032844343084597, 'reg_la

[INFO] Best LightGBM params: {'num_leaves': 398, 'max_depth': 16, 'learning_rate': 0.024443684742739523, 'min_child_samples': 79, 'reg_alpha': 0.09034051458018166, 'reg_lambda': 0.10249798605294153, 'colsample_bytree': 0.825862596791733, 'subsample': 0.8074096700218862}
[50]	valid_0's auc: 0.939503	valid_0's binary_logloss: 0.280872
[100]	valid_0's auc: 0.94622	valid_0's binary_logloss: 0.241381
[150]	valid_0's auc: 0.947968	valid_0's binary_logloss: 0.227247
[200]	valid_0's auc: 0.948934	valid_0's binary_logloss: 0.217277
[250]	valid_0's auc: 0.949206	valid_0's binary_logloss: 0.21137
[300]	valid_0's auc: 0.949522	valid_0's binary_logloss: 0.205379
[350]	valid_0's auc: 0.949895	valid_0's binary_logloss: 0.200227
[400]	valid_0's auc: 0.949942	valid_0's binary_logloss: 0.196964
[450]	valid_0's auc: 0.949895	valid_0's binary_logloss: 0.19508


In [26]:
evaluate_and_log(model_lgmb_reduced, X_test_reduced, y_test, experiment_name=EXPERIMENT_NAME, run_name="LGBM_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths)
evaluate_and_log(model_lgmb_reduced, X_test_reduced, y_test, experiment_name=EXPERIMENT_NAME, run_name="LGBM_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.1)
evaluate_and_log(model_lgmb_reduced, X_test_reduced, y_test, experiment_name=EXPERIMENT_NAME, run_name="LGBM_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.02)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9188558952898963), 'pr_auc': np.float64(0.566171140300614), 'precision': 0.3481135744846363, 'recall': 0.6606791338582677, 'f1': 0.4559735076844697, 'custom_loss': np.float64(1.2101466454431538)}
[INFO] Logged SHAP summary plot.


2025/12/31 00:56:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.
[INFO] Logged metrics: {'roc_auc': np.float64(0.9188558952898963), 'pr_auc': np.float64(0.566171140300614), 'precision': 0.11030280985723379, 'recall': 0.8954232283464567, 'f1': 0.1964107407907165, 'custom_loss': np.float64(0.6083584515866833)}
[INFO] Logged SHAP summary plot.


2025/12/31 00:56:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.
[INFO] Logged metrics: {'roc_auc': np.float64(0.9188558952898963), 'pr_auc': np.float64(0.566171140300614), 'precision': 0.05319034852546917, 'recall': 0.9763779527559056, 'f1': 0.10088477575511035, 'custom_loss': np.float64(0.6793104616114065)}
[INFO] Logged SHAP summary plot.


2025/12/31 00:56:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9188558952898963),
 'pr_auc': np.float64(0.566171140300614),
 'precision': 0.05319034852546917,
 'recall': 0.9763779527559056,
 'f1': 0.10088477575511035,
 'custom_loss': np.float64(0.6793104616114065)}

### 2.3. CatBoost

In [27]:
model_catoost_reduced, best_params_catoost_reduced, hist_df, plot_paths = train_catboost_optuna(X_train_reduced, y_train)

[I 2025-12-31 00:56:58,680] A new study created in memory with name: catboost_aucpr_optimization


[Optuna CatBoost Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-12-31 01:00:26,584] Trial 0 finished with value: 0.9136685291498262 and parameters: {'learning_rate': 0.02118663105117231, 'depth': 8, 'l2_leaf_reg': 2.364979904722989, 'subsample': 0.9340914433815872, 'border_count': 110}. Best is trial 0 with value: 0.9136685291498262.
[I 2025-12-31 01:02:27,360] Trial 1 finished with value: 0.886833307174299 and parameters: {'learning_rate': 0.013385812630271825, 'depth': 5, 'l2_leaf_reg': 4.652238378798031, 'subsample': 0.9801672710907207, 'border_count': 126}. Best is trial 0 with value: 0.9136685291498262.
[I 2025-12-31 01:03:40,962] Trial 2 finished with value: 0.8694997629823363 and parameters: {'learning_rate': 0.01208846846831615, 'depth': 3, 'l2_leaf_reg': 1.4425416910526934, 'subsample': 0.8690689339107842, 'border_count': 100}. Best is trial 0 with value: 0.9136685291498262.
[I 2025-12-31 01:05:14,490] Trial 3 finished with value: 0.8923758663236666 and parameters: {'learning_rate': 0.02870252059585402, 'depth': 4, 'l2_leaf_reg': 7

[INFO] Best CatBoost params: {'learning_rate': 0.028716492013710147, 'depth': 8, 'l2_leaf_reg': 3.2776603312909818, 'subsample': 0.8023562591008715, 'border_count': 33}
0:	test: 0.8409940	best: 0.8409940 (0)	total: 293ms	remaining: 24m 24s
100:	test: 0.9056012	best: 0.9056012 (100)	total: 26.7s	remaining: 21m 34s
200:	test: 0.9225995	best: 0.9226297 (199)	total: 53.1s	remaining: 21m 7s
300:	test: 0.9307788	best: 0.9307788 (300)	total: 1m 19s	remaining: 20m 39s
400:	test: 0.9350124	best: 0.9350124 (400)	total: 1m 45s	remaining: 20m 10s
500:	test: 0.9366784	best: 0.9367412 (492)	total: 2m 11s	remaining: 19m 42s
600:	test: 0.9394557	best: 0.9394607 (598)	total: 2m 37s	remaining: 19m 15s
700:	test: 0.9411823	best: 0.9411823 (700)	total: 3m 5s	remaining: 18m 59s
800:	test: 0.9423878	best: 0.9424898 (798)	total: 3m 32s	remaining: 18m 31s
900:	test: 0.9428982	best: 0.9431180 (890)	total: 3m 58s	remaining: 18m 4s
1000:	test: 0.9434565	best: 0.9435058 (988)	total: 4m 24s	remaining: 17m 36s
1100

In [28]:
evaluate_and_log(model_catoost_reduced, X_test_reduced, y_test, experiment_name=EXPERIMENT_NAME, run_name="CATBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths)
evaluate_and_log(model_catoost_reduced, X_test_reduced, y_test, experiment_name=EXPERIMENT_NAME, run_name="CATBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.1)
evaluate_and_log(model_catoost_reduced, X_test_reduced, y_test, experiment_name=EXPERIMENT_NAME, run_name="CATBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.02)

[INFO] Logged metrics: {'roc_auc': np.float64(0.913675918045998), 'pr_auc': np.float64(0.5385358367868641), 'precision': 0.2978939871807915, 'recall': 0.7204724409448819, 'f1': 0.4215072338587778, 'custom_loss': np.float64(1.020261116943814)}
[INFO] Logged SHAP summary plot.


2025/12/31 02:22:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.
[INFO] Logged metrics: {'roc_auc': np.float64(0.913675918045998), 'pr_auc': np.float64(0.5385358367868641), 'precision': 0.0757446301190216, 'recall': 0.937992125984252, 'f1': 0.14017024875438952, 'custom_loss': np.float64(0.6071984962915298)}
[INFO] Logged SHAP summary plot.


2025/12/31 02:22:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.
[INFO] Logged metrics: {'roc_auc': np.float64(0.913675918045998), 'pr_auc': np.float64(0.5385358367868641), 'precision': 0.039795043563211514, 'recall': 0.9822834645669292, 'f1': 0.07649121462377129, 'custom_loss': np.float64(0.8765028617875165)}
[INFO] Logged SHAP summary plot.


2025/12/31 02:22:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.913675918045998),
 'pr_auc': np.float64(0.5385358367868641),
 'precision': 0.039795043563211514,
 'recall': 0.9822834645669292,
 'f1': 0.07649121462377129,
 'custom_loss': np.float64(0.8765028617875165)}